<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/GRU-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import GRU, Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [ ]:
y_data = data.pop('4')
x_data = data

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0,shuffle=True)

In [ ]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [ ]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [ ]:
model = Sequential()
model.add(GRU(100,input_shape=(len(x_data.columns),1)))
model.add(Dense(4,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 100)               30900     
                                                                 
 dense_1 (Dense)             (None, 4)                 404       
                                                                 
Total params: 31,304
Trainable params: 31,304
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_x,train_y,epochs=6,batch_size=32)

Epoch 1/10
13256/13256 [==============================] - 54s 4ms/step - loss: 3.2221 - accuracy: 0.2386
Epoch 2/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2485
Epoch 3/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2511
Epoch 4/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2434
Epoch 5/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2551
Epoch 6/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2442
Epoch 7/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2506
Epoch 8/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2443
Epoch 9/10
13256/13256 [==============================] - 52s 4ms/step - loss: 3.2221 - accuracy: 0.2400
Epoch 10/10
13256/13256 [==============================

In [ ]:
score,accuracy = model.evaluate(test_x,test_y)
print('Score',score,"Accuracy",accuracy)

103560/103560 [==============================] - 238s 2ms/step - loss: 3.2214 - accuracy: 0.6278
Score 3.2213902473449707 Accuracy 0.6277751922607422
